In [1]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 336.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [5]:
import os
import json
from autogen import ConversableAgent

# put the fine-tune model to the LM studio, get the api key and base url , rewrite the config_list
llama3_config_list={"config_list":[{
    'model': 'meta/llama3-70b-instruct', #model here is your model name in the LM studio
    'api_key': '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',
    'base_url': "https://integrate.api.nvidia.com/v1",
    # 'api_version': '2024-02-15-preview',
    # 'api_type': 'azure',
}],
"cache_seed":None,
}


class ReportWriteReview():
    def __init__(self,report_keypoint):
        self.write_agent = ConversableAgent(
            "Jack (Analyst)",
            llm_config=llama3_config_list,
            system_message='''
            Your name is Jack, You are an equity research writer. Your task is to write a report on the company's financial performance and future prospects based on the provided company analysis brainstoming point information provided from your director. Your report should include a brief introduction of company, analysis of company's recent financial performance, how the recent event and news influence stock price and an analysis of the company's future prospects and risk. Your report should be written in English and be at least 300 words long. Also if receiving feedback from your director, you should be able to revise the report based on the feedback. The output should always follow the format: [Introduction of company]/n.../n/n[Analysis based on news and industry perspect]/n/n.../n/n[Analysis on Financial Report]/n/n.../n/n[Risk]/n/n.../n/n[Forecast]/n/n.../n/n[Recommendation]/n/n.../n
            '''
            )
        self.review_agent= ConversableAgent(

            "Emma(Director)",
            llm_config=llama3_config_list,
            system_message='''
            Your name is Emma, You are an professional equity research director. Your task is to review the report written by your analyst and provide feedback on the report. Your feedback should include detail suggestions for improvement, additional information that should be included, and any corrections that need to be made. Your feedback should be written in English and be at least 100 words long.
            '''
        )
        self.report_keypoint=report_keypoint

    def start_task(self):
        chat_result=self.review_agent.initiate_chat(self.write_agent,message="Jack, please write an equity research report for this company based on below information: "+self.report_keypoint,max_turn=3)
        return chat_result




if __name__=="__main__":
    report_keypoint='''[公司介绍]:\n\n腾讯控股是一家在软件服务行业的领先实体，自2004年06月16日成立并公开交易。截止今天，腾讯控股的总市值为3572736890364.0人民币，总股本数为9406890180.0，流通市值为3572736890364.0人民币，流通股数为9406890180.0。\n\n腾讯控股主要在中国运营，以股票代码00700在交易所进行交易。\n\n自2024-01-07至2024-01-14，腾讯控股的股票价格由288.80下跌至285.00，涨跌幅为：下跌1-2%。在此期间的公司新闻如下:\n\n[新闻标题]：腾讯控股00700.HK)连续33日回购 累计回购5978.00万股\n[新闻内容]：证券时报·数据宝统计，腾讯控股在港交所公告显示，1月8日以每股285.600港元至294.400港元的价格回购174.00\n\n如下所列为腾讯控股近期的一些金融基本面信息，记录时间为2023-12-31 00:00:00:\n\n[金融基本面]:\n\n报告期: 2023-12-31 00:00:00\n收入增长QOQ: 1.710497968345\n利润增长QOQ: 5.675512948548\n债务比例: 44.6071823926\n流水比例: 1.472201319298\n\n基于在2024-01-14之前的所有信息，让我们首先分析腾讯控股的积极发展和潜在担忧。请简洁地陈述，分别提出2-4个最重要的因素。大部分所提及的因素应该从公司的相关新闻中推断出来。那么让我们假设你对于下一周(2024-01-14至2024-01-21)的预测是下跌超过5%。提供一个总结分析来支持你的预测。预测结果需要从你最后的分析中推断出来，因此不作为你分析的基础因素。'''
    report_writer=ReportWriteReview(report_keypoint)
    chat_result=report_writer.start_task()
    print(chat_result)

Emma(Director) (to Jack (Analyst)):

Jack, please write an equity research report for this company based on below information: [公司介绍]:

腾讯控股是一家在软件服务行业的领先实体，自2004年06月16日成立并公开交易。截止今天，腾讯控股的总市值为3572736890364.0人民币，总股本数为9406890180.0，流通市值为3572736890364.0人民币，流通股数为9406890180.0。

腾讯控股主要在中国运营，以股票代码00700在交易所进行交易。

自2024-01-07至2024-01-14，腾讯控股的股票价格由288.80下跌至285.00，涨跌幅为：下跌1-2%。在此期间的公司新闻如下:

[新闻标题]：腾讯控股00700.HK)连续33日回购 累计回购5978.00万股
[新闻内容]：证券时报·数据宝统计，腾讯控股在港交所公告显示，1月8日以每股285.600港元至294.400港元的价格回购174.00

如下所列为腾讯控股近期的一些金融基本面信息，记录时间为2023-12-31 00:00:00:

[金融基本面]:

报告期: 2023-12-31 00:00:00
收入增长QOQ: 1.710497968345
利润增长QOQ: 5.675512948548
债务比例: 44.6071823926
流水比例: 1.472201319298

基于在2024-01-14之前的所有信息，让我们首先分析腾讯控股的积极发展和潜在担忧。请简洁地陈述，分别提出2-4个最重要的因素。大部分所提及的因素应该从公司的相关新闻中推断出来。那么让我们假设你对于下一周(2024-01-14至2024-01-21)的预测是下跌超过5%。提供一个总结分析来支持你的预测。预测结果需要从你最后的分析中推断出来，因此不作为你分析的基础因素。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 06-19 07:21:10]

Jack (Analyst) (to Emma(Director)):

Here is the equity research report:

**Introduction of Company**

Tencent Holdings Limited is a leading software service company founded on June 16, 2004, and listed publicly since then. As of today, the company has a total market capitalization of CNY 3,572,736,903,640 and a total share capital of 9,406,890,180 shares, with a floating market capitalization of CNY 3,572,736,903,640 and a floating share capital of 9,406,890,180 shares. The company primarily operates in China and is traded on the Hong Kong Stock Exchange under the stock code 00700.

**Analysis based on News and Industry Perspective**

Tencent Holdings' stock price declined by 1-2% from January 7 to January 14, 2024, with the company's recent news being a significant factor in this decline. The company's 33-day consecutive share repurchase, with a total of 59.78 million shares repurchased, is a positive sign, indicating the company's confidence in its financial performance. However, th

Emma(Director) (to Jack (Analyst)):

Here's my feedback on the report:

**Overall structure and clarity**: The report is well-structured, and the layout is easy to follow. However, some sections could be merged or reorganized for better flow. For instance, the "Analysis based on News and Industry Perspective" and "Analysis of Financial Report" sections could be combined into a single "Analysis" section.

**Introduction**: The introduction is concise, but it would be helpful to provide a brief overview of the company's main business segments and products. This will give readers a better understanding of Tencent Holdings' operations and help them appreciate the analysis that follows.

**Analysis**: The analysis is generally good, but it could be more detailed and specific. For example, you could discuss the implications of the company's 33-day consecutive share repurchase on its financials and investor sentiment. Additionally, you could provide more context on the industry perspective, s

Jack (Analyst) (to Emma(Director)):

Thank you for the detailed feedback! I'll make sure to revise the report accordingly. Here is the revised report:

**Introduction of Company**

Tencent Holdings Limited is a leading software service company founded on June 16, 2004, and listed publicly since then. The company operates primarily in China and is traded on the Hong Kong Stock Exchange under the stock code 00700. Tencent Holdings is a conglomerate with a diverse range of businesses, including social media, online gaming, e-commerce, and financial technology. Its main products and services include WeChat, QQ, Honor of Kings, and WeBank, among others.

**Analysis**

Tencent Holdings' stock price declined by 1-2% from January 7 to January 14, 2024, despite the company's 33-day consecutive share repurchase, which is a positive sign indicating the company's confidence in its financial performance. The share repurchase program, which amounts to 59.78 million shares, is expected to enhance sha

Emma(Director) (to Jack (Analyst)):

Excellent job on the revised report! You've addressed many of the concerns I raised earlier, and the report is now more comprehensive and detailed. Here are some specific things you did well:

* You added more context to the introduction, providing a better understanding of Tencent Holdings' businesses and products.
* You merged the analysis sections, creating a more cohesive and flowing narrative.
* You provided more specific and actionable insights, including a price target and valuation analysis.
* You addressed potential risks more comprehensively, including regulatory changes, cybersecurity threats, and competition.

Here are a few minor suggestions for further improvement:

* In the analysis section, you could provide more detail on how Tencent Holdings' share repurchase program will enhance shareholder value. For example, you could discuss how the program will reduce the company's outstanding shares and increase earnings per share.
* In the r

KeyboardInterrupt: 